In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import torch
import optuna
import os
import pytorch_lightning as pl

from Data.Drosophilla.FlyDataMod import FlyDataModule
from IPython.display import Image
from IPython.core.debugger import set_trace
from Models import BiLSTM as bl
from Models import LinearRegression as lrg
from Models import Transformer as tfm
from pytorch_lightning import LightningDataModule
from sklearn import preprocessing
from torch import nn as nn
from Utils import evaluations as ev
from Utils import callbacks as cb
from Utils import loggers as lg

In [ ]:
label_val=10
label_type="gamma"
dm    = FlyDataModule(cell_line="S2",
                  data_win_radius=5,
                  batch_size=1,
                  label_type=label_type,
                  label_val=label_val)
dm.setup()

In [ ]:
#regression
!mkdir Experiments/Benchmark_Architectures/Regression

trainer = pl.Trainer(
    gpus=1,
    max_epochs=50,
    callbacks=[cb.getcb()],
    default_root_dir="Experiments/Benchmark_Architectures/Regression",
    terminate_on_nan=True)

model_linear = lrg.LinearRegressionModule(
    inputSize=29,
    outputSize=1,
    hasRidge=True)
model_linear.cuda()
#trainer.fit(model_linear, dm)

In [ ]:
#bilstm
!mkdir Experiments/Benchmark_Architectures/LSTM

trainer = pl.Trainer(
    gpus=1,
    max_epochs=50,
    callbacks=[cb.getcb()],
    default_root_dir="Experiments/Benchmark_Architectures/LSTM",
    terminate_on_nan=True)

model_lstm =  bl.BiLSTMModule(
            input_size=29,
            hidden_size=64,
            num_layers=5,
            lr=0.0001,
            dropout=0.2)
model_lstm.cuda()
#trainer.fit(model_lstm, dm)
print("done trianing")

In [ ]:
#transformer 
#bilstm
!mkdir Experiments/Benchmark_Architectures/Transformer
trainer = pl.Trainer(
    gpus=1,
    max_epochs=50,
    callbacks=[cb.getcb()],
    default_root_dir="Experiments/Benchmark_Architectures/Transformer",
    terminate_on_nan=True)

model_tfm  = tfm.TransformerModule(
                ntoken=1,
                ninp=29,
                nhead=1,
                nhid=2048,
                nlayers=1,
                dropout=0,
                optimi="Adam",
                lr=.001)
model_tfm.cuda()
#trainer.fit(model_tfm, dm)
#print("done training Transformer")

In [ ]:
from Utils import evaluations as eva

model_linear.cuda()
linear_path = glob.glob("Experiments/Benchmark_Architectures/Regression/lightning_logs/version_0/checkpoints/*")[0]
model_linear = model_linear.load_from_checkpoint(linear_path).cuda()
print(eva.getModelMetrics(model_linear, dm, 'test'))
ev.createPlot(model_linear, dm, 'test', 'morgarbage.png')

model_lstm.cuda()
lstm_path = glob.glob("Experiments/Benchmark_Architectures/LSTM/lightning_logs/version_0/checkpoints/*")[0]
model_lstm = model_lstm.load_from_checkpoint(lstm_path).cuda()
print(eva.getModelMetrics(model_lstm, dm, 'test'))
ev.createPlot(model_lstm, dm, "test", "garbage.png")

model_tfm.cuda()
tfm_path = glob.glob("Experiments/Benchmark_Architectures/Transformer/lightning_logs/version_0/checkpoints/*")[0]
model_tfm = model_tfm.load_from_checkpoint(tfm_path).cuda()
print(eva.getModelMetrics(model_tfm, dm, 'test'))
ev.createPlot(model_tfm, dm, 'test', 'moregarbage.png')